In [31]:
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv
import os
from langgraph.prebuilt import tool_node
import google.generativeai as genai
import fitz
from typing import TypedDict
from pprint import pprint
import json

In [32]:
class AgentType(TypedDict):
    resume_text: str
    preference_alignment: dict

In [33]:
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text() for page in doc)
    return text

resume_text = extract_text_from_pdf("resume.pdf")
print(resume_text[:1000])

Software Engineer
vipshalgw@gmail.com
https://www.linkedin.com/in/vipshal
+919524045269
Coimbatore, Tamil Nadu, 641035, India
I'm a passionate Full Stack Developer with over 1+ years of experience, specializing in MERN and PERN stack 
development. I thrive in dynamic environments, leveraging my skills to build scalable web applications and deliver 
impactful solutions. Always eager to learn and grow, I aim to contribute to innovative projects that make a difference.
Work Experience
Projects
Vipshal WG
Software Engineer
SNS Square | Coimbatore
 - 
Worked in a fast-paced startup environment for over 1+ years, gaining hands-on experience with the MERN and PERN 
stacks. Contributed to building scalable web applications by working across both frontend and backend development. 
Integrated technologies such as Elasticsearch for advanced search capabilities and RabbitMQ for reliable 
asynchronous communication.
Nov 2024
Present
Square Bridge
 - 
This was the tool which will bridge student, pla

In [35]:
def candidate_preference_agent(state):
    resume_text = state["resume_text"]
    prompt = f"""
    You are a career guidance AI assistant. A student's resume is provided below.
    Please extract their job-related preferences in this structured JSON format:

    {{
      "preferred_location": "...",
      "interests": ["...", "..."],
      "skills": ["...", "..."],
      "expected_ctc": "...",
      "career_goals": "..."
    }}

    Resume Text:
    {resume_text}
    """

    response = model.generate_content(prompt)
    return {"preference_alignment": response.text}


In [36]:
builder = StateGraph(AgentType)


builder.add_node("CandidatePreference", candidate_preference_agent)
builder.set_entry_point("CandidatePreference")
builder.set_finish_point("CandidatePreference")

graph = builder.compile()


In [37]:
result = graph.invoke({
    "resume_text": resume_text
})

json_string = result["preference_alignment"].strip().replace('```json\n', '').replace('\n```', '')
formatted_dict = json.loads(json_string)
pprint(formatted_dict)


{'career_goals': 'Contribute to innovative projects that make a difference',
 'expected_ctc': None,
 'interests': ['Full Stack Development',
               'MERN Stack',
               'PERN Stack',
               'Scalable Web Applications',
               'Innovative Projects'],
 'preferred_location': 'Coimbatore, Tamil Nadu, India',
 'skills': ['JavaScript',
            'Nodejs',
            'fastAPI',
            'Python',
            'Express js',
            'MongoDB',
            'PostgreSQL',
            'Redis',
            'Elasticsearch',
            'RabbitMQ',
            'React js',
            'Next js',
            'Amazon S3',
            'AWS Elastic Beanstalk',
            'AWS Cloudfront',
            'Prisma ORM',
            'UI/UX',
            'Responsive Web Design']}
